<a href="https://colab.research.google.com/github/jasonlee87/AIDL-Lab/blob/master/03_01_char_rnn_high_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RNN 기반 Text Genration

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

위 출력은 모델이 생성한 문장이다. "Q"로 시작하는 곳부터 Text Generation 한 경우이다.
생성된 문장을 보면 일부 문장은 문법적 맞지만 대부분은 의미 파악이 어렵다. 하지만 훈련을 처음 시작했을때 모델은 알파벳도 그리고 이것이 텍스트의 기본 단위인지도 몰랐다는 점에 비춰보면 유의미한 결과라 할수 있다.

이 예제를 통해 구현 과정을 살펴보자

## Setup

### TensorFlow와 관련 모듈들을 import 한다

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

### 세익스피어 데이타셋을 다운로드한다

In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### Data 읽기

하나의 텍스트로 묶는다

In [0]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [0]:
# Take a look at the first 250 characters in text
print(text[:250])
#print(text[len(text)-100:len(text)])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [0]:
print (type(vocab))
print (len(vocab))
print (vocab)

<class 'list'>
65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## 텍스트 가공

### 텍스트를 벡터화한다

학습하기 전에 문자를 숫자로 표기할 필요가 있다. 문자를 인덱스로, 인덱스를 문자로 맵핑한 2개의 lookup 테이블을 만든다. 

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

#print (type(char2idx)) # class 'dick'
#print (type(idx2char)) # class 'numpy.ndarray'
#print (type(text))     # class 'str'

text_as_int = np.array([char2idx[c] for c in text])
#print (text_as_int)     # [18 47 56 ... 45 8 0]

이제 각 문자마다 고유의 인덱스를 두었고, 이 예에서의 인덱스 범위는 0~64가 된다.

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


예를 들어 첫 13문자가 어떻게 숫자로 표현되었는지 보자

In [0]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


### 예측하기

일련의 캐릭터(시퀀스)가 주어졌을때 다음 캐릭터로 나올 가장 높은 확률의 문자는 뭘까? 우리가 할 과제가 바로 이를 수행할 모델을 학습 시키는것이다. 입력으로 문자열이 들어가면 모델은 그에 뒤따르는 문자열을 출력(예측)하는 것이다.

RNN은이 앞서 시퀀스로부터 계산된 내부 상태(정보)를 예측에 활용한다.


### 학습 예(training example) 만들기

그런 다음 텍스트를 예제 시퀀스로 나눕니다. 각 입력 시퀀스마다 seq_length 문자를  포함한다. 출력 시퀀스 길이는 한 문자 오른쪽으로 이동한것을 제외하곤 입력 시퀀스 길이와 같다
우린 Text를 'seq_length+1' 크기로 자를 것이다. 예를 들어 텍스트가 "Hello"라면 seq_length = 4 가 된다. 따라서 입력 시퀀스는 "Hell"(길이 4)이고 기대하는 출력 시퀀스도 길이 4인 "ello"가 되도록 했다.

이 작업을 위해 tf.data.Dataset.from_tensor_slices 함수를 사용하여 텍스트 벡터를 일련의 문자 인덱스로 변환했다.

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
print (char_dataset)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

<TensorSliceDataset shapes: (), types: tf.int64>
F
i
r
s
t


각각의 문자를 원하는 크기의 시퀀스로 변환할때 batch method를 쓰면 편하다.

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

#print (type(sequences))

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()]))) 

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


입출력 시퀀스를 map method로 묶어 dataset을 만든다

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

입력 시퀀스와 타겟 시퀀스 실제 모습(한 문자 shift 되었고 길이는 같음)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


이들 벡터의 각 인덱스가 매 타임스텝마다 처리된다. 타임스텝 0에서 'F'에 해당하는 인덱스를 받고, 다음 문자로 "i"가 출력되게끔 예측을 시도한다. 다음 타임스텝도 같은 일을 하겠지만 'RNN'은 이제부턴 다음의 입력 문자뿐만 아니라 문맥(지나온 동안의 시퀀스 내부 정보)을 함께 고려한다.

In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


### 학습 배치 만들기

우리는 텍스트를 운용가능한 시퀀스로 자르기 위해 'tf.data'를 사용할것이다. 이 데이타를 모델에 넣기(feeding)전에 먼저 데이타를 섞고, 배치단위로 취할것이다. 

In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## 모델 설계

'tf.keras.Squential'을 이용해서 모델을 정의했으며 이 예제에선 간략히 3개층으로 모델을 정의했다.
* `tf.keras.layers.Embedding`: 입력층. 문자와 `embedding_dim` dimensions을 가진 문자 벡터가 연결하는 테이블을 가지고 있다
* `tf.keras.layers.GRU`: `units=rnn_units` 크기를 가진 GRU 층
* `tf.keras.layers.Dense`: `vocab_size` 출력을 가진 출력층.

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
print (vocab_size)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

65


In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

아래 그림은 문자가 입력되면 lookup table에 의해 입력 문자에 대응하는 embedding vector가 GRU의 입력으로 들어간다. 그리고 dense layer를 거처 다음 문자를 예측할수 있는 logit을 만드는 과정을 설명하고 있다.
![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

## 모델 예비 시도

모델을 한번 실행해 보자 우리 예측대로 동작하는지. 그리고 먼저 출력의 shape를 체크해 보자

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


(위 예에서 시퀀스 길이를 100으로 설정했지만 이 모델은 어떠한 시퀀스 길이도 가능하다)

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


모델에서 실제 예측을 하가 위해선 출력 분포에서 인덱스를 샘플링해야 합니다. 이 분포는 문자 어휘에 대한 로짓(logit)으로 정의됩니다.

배치 방식의 첫 번째 예를 봅시다.



In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

매 timestep 마다 이렇게 입력을 받아 다음 문자를 예측하는것이다.

In [0]:
sampled_indices

array([59, 21,  3,  0, 22, 20, 63, 47, 48, 11, 51,  9, 26, 62, 37, 48, 52,
       31, 13,  2, 46, 11, 55,  9, 26, 41, 26, 63, 28,  2, 60, 22,  6,  1,
       20, 46, 54, 57, 41, 38, 21, 28, 38, 52,  8, 60, 14, 57, 33, 30, 28,
       21, 25, 29, 41, 58, 60, 22, 11, 22, 44,  0, 55, 59, 14,  8, 38, 29,
       38, 51, 54, 37, 54, 39,  9, 60, 20, 57, 51, 34, 19, 22, 37,  4, 28,
       53, 43, 23,  7, 38, 43,  0,  1, 50,  9, 49, 63, 43,  6,  9])

학습되지 않은 모델로 결과를 한번 보자.

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'gainst our meaning, have prevented:\nBecause, my lord, we would have had you heard\nThe traitor speak,'

Next Char Predictions: 
 'uI$\nJHyij;m3NxYjnSA!h;q3NcNyP!vJ, HhpscZIPZn.vBsURPIMQctvJ;Jf\nquB.ZQZmpYpa3vHsmVGJY&PoeK-Ze\n l3kye,3'


## 모델 학습

이제부턴 이 문제를 표준 분류 문제로 간주할수있다. 즉 앞서 주어진 RNN 상태와 매 timestep 마다의 입력으로 새로운 문자의 클래스를 예측하는 분류 문제로 볼수 있는것이다.

### 옵티마이저와 손실함수 선택



여기선 표준 `tf.keras.losses.sparse_categorical_crossentropy` 손실함수를 이용할 것이다. 
그리고 우리 모델이 logits를 리턴하기 때문에 'from_logits' flag 를 활성화시켰다. 


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.175708


그리고 `tf.keras.Model.compile` method로 학습 설정들을 정의하는데 여기선 옵티마이저로 `tf.keras.optimizers.Adam`을 사용할 것이다.

In [0]:
model.compile(optimizer='adam', loss=loss)

### 체크포인트(파라미터) 저장 설정

학습중 체크포인트를 저장하기 위해 `tf.keras.callbacks.ModelCheckpoint` 메소드를 사용한다.

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### 학습 실행

여기선 10 epochs 학습시켰다. 대략 2:30분 소요되었다.

In [0]:
EPOCHS=10

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 972s 6s/step - loss: 2.6594
Epoch 2/10
172/172 [==============================] - 1021s 6s/step - loss: 1.9528
Epoch 3/10
172/172 [==============================] - 961s 6s/step - loss: 1.6842
Epoch 4/10
172/172 [==============================] - 940s 5s/step - loss: 1.5380
Epoch 5/10
172/172 [==============================] - 937s 5s/step - loss: 1.4514
Epoch 6/10
172/172 [==============================] - 979s 6s/step - loss: 1.3933
Epoch 7/10
172/172 [==============================] - 1006s 6s/step - loss: 1.3478
Epoch 8/10
172/172 [==============================] - 1013s 6s/step - loss: 1.3101
Epoch 9/10
172/172 [==============================] - 1031s 6s/step - loss: 1.2757
Epoch 10/10
172/172 [==============================] - 990s 6s/step - loss: 1.2436


## 텍스트 생성

### 최신 체크포인트 가져오기

예측 과정을 들여다보기 위해 배치 크기를 1로 한다.
이를 위해 'batch_size'로 재설계하고 체크포인트를 불러온다


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


### 예측 과정

다음 코드 블럭이 텍스트를 생성한다.

* 사전 약속한 시작 문자열을 만나면 예측을 시작한다. RNN 상태를 초기화하고 생성할 문자의 갯수를 정한다.

* 시작 문자열과 RNN 상태를 이용 다음 문자의 예측 분포를 구한다.

* 그리고 예측 문자의 인덱스를 구하기 위해 categorical distribution을 이용한다. 예측한 문자는 다음 단계의 입력으로 쓰인다.

* RNN 상태는 다시 모델로 피드백되면서 더 많은 문맥을 가지게 된다. 다음 문자를 예측한 후엔 RNN 상태가 갱신되고 이는 다시 모델로 피드백된다. 앞서 예측 단어를 통해 문맥 정보를 얻어내는것 이것이 학습의 방법이다.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

생성된 글을 보면 모델이 학습량 부족에도 불구하고 언제 대문자를 쓰는지, 언제 단락을 만드는지, 세익스피어처럼  글쓰기를 흉내 내고 있음을 볼 수 있다.

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: my lord.
An hateou-latence that kiss my lord's death,
knop we may could think no less chamberlage?
O blawlich, go
ather, wherefore has bore me:

Second Senatarion:
Made her than my soul profess'd to their appear?
And I send the back of known in
averenh peins. O, if I wounger his heart.
Look, he hath a very crack'd in men,
In brief,--
Injeing, but
Off the good god-hast be a thousand men than she will privare
A great dear Jap, may far in thy contract,
Unless I am stugh 'mongst our studment.

Second Senator:
So many counser take my lord;
I have from the city.

GREGO:
Good, let my lady else?--but did not take off thou now,--
Then if you well drunk both your yields to-night:
A brainn mystery of this death?

Second MusiBis men and unge-up the war;
How was this cause that got delight, then bloody;
One of this face of my husband's creatur-vicans that thou art a craf
Thus faults to hours must not be gone;
Poor brother--leapt in eyes, great Autiul father!
The lobber that no more than cont

성능을 올릴수 있는 가장 쉬운 방법은 학습량을 더 늘리는것이며(epochs=30) 또한 다른 RNN 층을 쓴다거나 예측 관련한 temperature 파라미터를 조정하는 방법도 가능할 것이다.



## 고급: 맞춤형 학습

이 예는 간단한 반면 여러 제어 기능을 제공하지는 않습니다.

앞의 예를 통해 모델을 수동으로 실행하는 방법을 살펴 봤으므로 이번에 학습 루프를 풀고 직접 구현하는 방법을 알아보고자 합니다. 예를 들어, 모델의 개방 루프 출력을 안정화하기 위해 커리큘럼 학습을 구현하는 경우 이 예제가 시작점이 될것입니다.

tf.GradientTape를 사용하여 그라디언트를 추적합니다. Eager execution guide를 읽으면이 접근 방식에 대해 자세히 알아볼 수 있습니다.

절차는 다음과 같이 작동합니다.

먼저 RNN 상태를 초기화하십시오. tf.keras.Model.reset_states 메소드를 호출하여 이를 수행합니다.

다음으로 데이터 집합을 반복하고 각 집합과 관련된 예측을 계산합니다.

tf.GradientTape를 열고 해당 컨텍스트에서 예측 및 손실을 계산하십시오.

tf.GradientTape.grads 방법을 사용하여 모델 변수와 관련하여 손실의 기울기를 계산하십시오.

마지막으로 옵티마이저의 tf.train.Optimizer.apply_gradients 메소드를 사용하여 한 단계 아래로 이동하십시오.


In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [0]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.174903392791748
